In [11]:

import cv2
import easyocr
import numpy as np
from PIL import ImageGrab, Image
import time

def wait_before_screenshot(seconds=3):
    print(f"Waiting {seconds} seconds before taking the screenshot...")
    time.sleep(seconds)

def capture_full_screenshot():
    screenshot = ImageGrab.grab()
    return np.array(screenshot)

def convert_to_grayscale(image_np):
    return cv2.cvtColor(image_np, cv2.COLOR_BGR2GRAY)

def initialize_ocr_reader(languages=['en', 'es', 'pt', 'it', 'fr']):
    return easyocr.Reader(languages, gpu=False)

def perform_ocr(image_np, reader):
    return reader.readtext(image_np)

def detect_chat_section(ocr_result):
    keywords = ["chat", "comments", "live chat", "live", "users", "message", "comentarios", "en vivo"]

    for (bbox, text, prob) in ocr_result:
        text_lower = text.lower()
        for keyword in keywords:
            if keyword in text_lower:
                print(f"Detected keyword: '{keyword}' in text: '{text}'")
                return bbox

    return None

def crop_comment_section(screenshot_np, bbox, roi_left, roi_right, screen_width, screen_height):
    (top_left, top_right, bottom_right, bottom_left) = bbox
    left = int(min(top_left[0], bottom_left[0])) + roi_left
    top = int(min(top_left[1], top_right[1]))

    new_bottom = min(top + 600, screen_height)
    new_left = max(left - 20, roi_left)
    new_right = min(left + 800, screen_width)

    return screenshot_np[top:new_bottom, new_left:new_right]

def save_image(image_np, file_name):
    Image.fromarray(image_np).save(file_name)
    print(f"Screenshot saved as {file_name}.")

def extract_comments(ocr_result):
    comments_dict = {}
    lines = []

    print("OCR result for extracted section:")
    for (bbox, text, prob) in ocr_result:
        print(f"Detected text: '{text}' with confidence {prob}")
        lines.append((bbox, text))

    # Agrupamos los comentarios y los nombres de usuarios según la distancia vertical
    previous_text = ""
    previous_y = None

    for (bbox, text) in lines:
        # Calculamos la posición Y de la caja de la detección
        current_y = (bbox[0][1] + bbox[2][1]) / 2

        # Si la posición Y es cercana a la anterior, asumimos que es parte del mismo comentario
        if previous_y is not None and abs(current_y - previous_y) < 20:
            previous_text += " " + text  # Agregamos al comentario anterior
        else:
            # Guardamos el comentario si ya hay un nombre y comentario
            if previous_text:
                comments_dict[previous_text.split(" ")[0]] = previous_text[len(previous_text.split(" ")[0]):].strip()
            previous_text = text  # Comenzamos una nueva entrada

        previous_y = current_y

    # Añadir el último comentario
    if previous_text:
        comments_dict[previous_text.split(" ")[0]] = previous_text[len(previous_text.split(" ")[0]):].strip()

    return comments_dict

def take_screenshot_of_comment_section(file_name):
    wait_before_screenshot()

    screenshot_np = capture_full_screenshot()
    gray_screenshot = convert_to_grayscale(screenshot_np)

    reader = initialize_ocr_reader()

    screen_width = screenshot_np.shape[1]
    screen_height = screenshot_np.shape[0]
    roi_left = int(screen_width * 0.5)
    roi_right = screen_width

    cropped_screenshot = screenshot_np[:, roi_left:roi_right]
    gray_cropped = convert_to_grayscale(cropped_screenshot)

    ocr_result_cropped = perform_ocr(gray_cropped, reader)

    keyword_bbox = detect_chat_section(ocr_result_cropped)

    if keyword_bbox:
        cropped_comment_section = crop_comment_section(screenshot_np, keyword_bbox, roi_left, roi_right, screen_width, screen_height)
        save_image(cropped_comment_section, file_name)

        comments_dict = extract_comments(ocr_result_cropped)
        return comments_dict
    else:
        print("No comment section detected.")
        return {}

# Example usage
comments = take_screenshot_of_comment_section("comments_section_auto.png")
print(comments)


    

Waiting 3 seconds before taking the screenshot...


Using CPU. Note: This module is much faster with a GPU.


Detected keyword: 'chat' in text: 'Chat destacado'
Screenshot saved as comments_section_auto.png.
OCR result for extracted section:
Detected text: 'rubius_awaiting - Twitch' with confidence 0.8221718829342818
Detected text: '"P Home' with confidence 0.3536012249027657
Detected text: 'WooCommerce' with confidence 0.9987771238079669
Detected text: '(27) Learn Italian wit.' with confidence 0.8284670715914711
Detected text: 'D Todos los marcadores' with confidence 0.8136375157113005
Detected text: 'Chat destacado' with confidence 0.9568096604962775
Detected text: 'Lofi Girl' with confidence 0.6942946909367148
Detected text: 'New Back-to-School' with confidence 0.9887421846956596
Detected text: 'Maram   denji ok good' with confidence 0.8045780898957936
Detected text: 'NDragon89' with confidence 0.7111622744398246
Detected text: 'Eveningtimes' with confidence 0.9440361485579949
Detected text: 'ok give south korea' with confidence 0.9781932567507707
Detected text: 'ABGOD' with confidence 0.44

In [8]:
text

NameError: name 'text' is not defined